In [1]:
import os

# Función para instalar OpenJDK 8 si no está instalado previamente.
def install_java():
    # Verifica si la ruta del JDK existe.
    if not os.path.exists('/usr/lib/jvm/java-8-openjdk-amd64'):
        print("Instalando OpenJDK 8...")
        # Comando para instalar Java en modo silencioso.
        !apt-get install openjdk-8-jdk-headless -qq > /dev/null
        print("OpenJDK 8 instalado correctamente.")
    else:
        print("OpenJDK 8 ya está instalado.")



# Función para descargar Apache Spark solo si no está ya descargado.
def download_spark():
    # URL y nombre del archivo comprimido de Spark.
    spark_url = "https://archive.apache.org/dist/spark/spark-3.4.3/spark-3.4.3-bin-hadoop3.tgz"
    spark_tar = "spark-3.4.3-bin-hadoop3.tgz"

    # Verifica si el archivo comprimido ya existe.
    if not os.path.exists(spark_tar):
        print("Descargando Apache Spark...")
        # Comando para descargar el archivo desde la URL.
        !wget -q $spark_url
        print("Descarga completa.")
    else:
        print("El archivo de Apache Spark ya está descargado.")



# Función para descomprimir el archivo de Apache Spark solo si no está descomprimido.
def extract_spark():
    # Nombre de la carpeta descomprimida.
    spark_dir = "spark-3.4.3-bin-hadoop3"
    spark_tar = "spark-3.4.3-bin-hadoop3.tgz"

    # Verifica si la carpeta descomprimida ya existe.
    if not os.path.exists(spark_dir):
        print("Descomprimiendo Apache Spark...")
        # Comando para descomprimir el archivo.
        !tar xf $spark_tar
        print("Apache Spark descomprimido.")
    else:
        print("La carpeta de Apache Spark ya existe.")



# Función para configurar las variables de entorno necesarias para Spark.
def set_environment_variables():
    # Establece la ruta de JAVA_HOME y SPARK_HOME.
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    os.environ["SPARK_HOME"] = "/content/spark-3.4.3-bin-hadoop3"
    print("Variables de entorno configuradas.")



# Función para verificar si un paquete de Python ya está instalado.
import subprocess
import sys

def is_package_installed(package_name):
    try:
        # Ejecuta el comando `pip show` para verificar si el paquete está instalado.
        subprocess.check_call(
            [sys.executable, "-m", "pip", "show", package_name],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL
        )
        return True
    except subprocess.CalledProcessError:
        return False



# Función para instalar librerías de Python necesarias (findspark y pyspark).
def install_python_libraries():
    # Verifica e instala findspark.
    if not is_package_installed("findspark"):
        print("Instalando findspark...")
        !pip install -q findspark
    else:
        print("findspark ya está instalado.")

    # Verifica e instala pyspark.
    if not is_package_installed("pyspark"):
        print("Instalando pyspark...")
        !pip install -q pyspark
    else:
        print("pyspark ya está instalado.")



install_java()                # Instala OpenJDK 8
download_spark()              # Descarga Apache Spark
extract_spark()               # Descomprime Apache Spark
set_environment_variables()   # Configura las variables de entorno
install_python_libraries()    # Instala las librerías de Python

OpenJDK 8 ya está instalado.
El archivo de Apache Spark ya está descargado.
La carpeta de Apache Spark ya existe.
Variables de entorno configuradas.
findspark ya está instalado.
pyspark ya está instalado.


---

# **Crear una sesion en Spark**

In [2]:
# Importar la biblioteca findspark, que ayuda a configurar PySpark correctamente en el entorno de ejecución
import findspark

# Inicializar findspark para establecer las variables necesarias de PySpark
findspark.init()

# Importar SparkSession desde la biblioteca pyspark.sql
# SparkSession es la entrada principal para trabajar con DataFrames en PySpark
from pyspark.sql import SparkSession

# Crear o obtener una SparkSession
# Esto inicializa un entorno de Spark si no existe uno, o reutiliza uno existente
spark = SparkSession.builder.getOrCreate()

# Obtener el contexto de Spark (SparkContext) desde la SparkSession
# SparkContext es la entrada principal para trabajar con RDDs en PySpark
sc = spark.sparkContext


---

### **Teoría sobre las Variables Broadcast en Spark**

En Spark, las variables **Broadcast** son una herramienta importante para optimizar el rendimiento de las aplicaciones distribuidas cuando se necesitan compartir datos de manera eficiente entre las distintas particiones de un clúster. Una **variable Broadcast** es un mecanismo que permite distribuir de manera eficiente y en solo una vez, grandes conjuntos de datos o variables a todos los nodos de un clúster, sin necesidad de enviarlos repetidamente durante cada tarea.

### ¿Qué son las Variables Broadcast?

Las variables Broadcast son **datos de solo lectura** que son compartidos entre todas las tareas en todas las particiones de los nodos de un clúster. Son útiles cuando tienes un conjunto de datos pequeño o mediano que se necesita en todas las tareas de un trabajo, como una tabla de búsqueda, una lista de configuraciones o parámetros, o un modelo entrenado que debe ser utilizado por cada nodo del clúster.

### Beneficios de las Variables Broadcast

1. **Reducción del Coste de Comunicación**:
   - Sin las variables broadcast, si un conjunto de datos es necesario en varias particiones, Spark tendría que enviar una copia de ese conjunto de datos a cada partición de forma repetida. Esto genera una gran sobrecarga de red. Al usar variables Broadcast, el conjunto de datos solo se envía una vez y se distribuye de manera eficiente a todos los nodos.
   
2. **Mejora del Rendimiento**:
   - Las variables Broadcast permiten que los datos sean **inmutables** y sean **solo lectura**, lo que hace que puedan ser fácilmente cacheados y compartidos entre los nodos sin necesidad de replicarlos varias veces, mejorando así el rendimiento en trabajos distribuidos.

3. **Uso Eficiente de la Memoria**:
   - Al ser distribuidas solo una vez, se minimiza el uso de memoria en cada nodo y se optimiza el proceso de ejecución.

### ¿Cómo Funcionan las Variables Broadcast?

1. **Creación**:
   - Una variable Broadcast se crea utilizando el método `sc.broadcast()`, donde `sc` es el `SparkContext` que inicializa la sesión de Spark.
   
2. **Acceso a los Datos**:
   - Una vez que se crea la variable Broadcast, el acceso a los datos se realiza a través del método `.value`, que devuelve el contenido de la variable.

3. **Distribución**:
   - Una vez que la variable es "broadcasted", todos los nodos del clúster pueden acceder a esa variable de forma local sin necesidad de acceder a los datos centralmente.

### Ejemplo de Uso:

In [3]:
# Crear un conjunto de datos a compartir entre las tareas
broadcast_var = sc.broadcast([1, 2, 3, 4, 5, 6])

# Usar la variable broadcast en una función que se aplica a cada partición
rdd = sc.parallelize([10, 20, 30, 40])
result = rdd.map(lambda x: x + sum(broadcast_var.value))

# Mostrar el resultado
print(result.collect())

[31, 41, 51, 61]



En este ejemplo, la lista `[1, 2, 3, 4]` se distribuye a todas las particiones, y luego es utilizada dentro de una operación `map()` sobre el RDD sin que sea necesario enviarla repetidamente a cada partición.

### Casos de Uso Comunes:
- **Tablas de búsqueda**: Cuando necesitas usar una tabla estática o un conjunto de parámetros pequeños en cada tarea de Spark.
- **Modelos entrenados**: Un modelo de machine learning preentrenado que debe ser utilizado en todas las particiones sin necesidad de enviarlo repetidamente.
- **Datos estáticos o de configuración**: Información que no cambia durante el procesamiento y que debe ser accesible en todos los nodos.

### Consideraciones:
- Las variables Broadcast deben ser **inmutables** (no deben modificarse una vez que se han "broadcasted").
- **Solo deben usarse para conjuntos de datos pequeños a medianos**: Si la cantidad de datos es muy grande, el uso de Broadcast puede no ser eficiente y podría consumir mucha memoria en los nodos.

Las variables Broadcast son una forma de optimizar el uso de recursos en aplicaciones distribuidas, mejorando la eficiencia de las operaciones que requieren compartir información entre varias tareas y particiones.

# **Uso de unpersist() para Liberar Memoria en PySpark**

El método `unpersist()` se utiliza cuando **has persistido un RDD** o **DataFrame** en memoria o en disco y deseas liberar los recursos que han sido reservados para ello.

En PySpark, puedes persistir datos de un RDD o DataFrame en memoria utilizando el método `.persist()`, lo que significa que los datos serán almacenados en memoria y se evitará el re-calculo de esos datos cuando se vuelvan a utilizar. Si ya no necesitas esos datos en memoria y deseas liberar la memoria, puedes usar `unpersist()`.

### Ejemplo práctico con `unpersist()`:

In [4]:
# Eliminar la variable Broadcast
broadcast_var.unpersist()

### Explicación:

- **`broadcast_var.unpersist()`**: Este método se usa para liberar la memoria en los nodos del clúster donde la variable Broadcast ha sido almacenada. Al llamar a `unpersist()`, estamos indicando que ya no necesitamos más la variable Broadcast y queremos liberar los recursos asociados con ella.

### ¿Por qué se debe hacer?

- **Liberar recursos**: Las variables Broadcast se distribuyen a todos los nodos del clúster, lo que implica que ocupan memoria en cada uno de esos nodos. Si ya no necesitamos esa variable, es recomendable liberarla para evitar que ocupe memoria innecesaria.
- **Eficiencia**: Si no se hace esto, la variable Broadcast continuará ocupando memoria incluso después de que ya no sea necesaria, lo que podría afectar el rendimiento de otras tareas o procesos que requieren memoria.
- **Práctica recomendada**: Es una buena práctica liberar cualquier recurso, como variables Broadcast, cuando ya no son necesarios para asegurar un uso eficiente de la memoria y evitar fugas de memoria.

De esta forma, aseguras que los recursos se gestionan adecuadamente en el clúster.

# **Código para eliminar una variable Broadcast en PySpark**

Para eliminar una variable Broadcast, usamos el método `destroy()` que está disponible para las variables broadcast. Aquí tienes el código:

### Código para destruir la variable Broadcast:

In [5]:
# Destruir la variable Broadcast
broadcast_var.destroy()

### Explicación:

- **`broadcast_var.destroy()`**: Este método destruye la variable Broadcast de manera explícita. Al hacerlo, se libera cualquier referencia interna que Spark pueda estar utilizando para la variable, asegurando que los datos asociados con ella sean eliminados de manera más completa.

### ¿Por qué se debe hacer?

- **Liberación completa de recursos**: Aunque `unpersist()` libera la memoria, `destroy()` asegura que la variable Broadcast sea eliminada por completo, lo que significa que no quedará ninguna referencia en el sistema, liberando todos los recursos asociados.
- **Evitar fugas de memoria**: Al destruir la variable Broadcast, te aseguras de que no quede memoria ocupada innecesaria en los nodos del clúster. Esto es especialmente importante en entornos de producción donde se maneja una gran cantidad de datos.
- **Práctica recomendada**: Llamar a `destroy()` es útil cuando sabes que la variable Broadcast ya no se va a utilizar en el futuro, asegurando una limpieza completa y eficiente de los recursos.

Usar `destroy()` es una medida más fuerte que `unpersist()`, ya que garantiza que la variable y los datos asociados sean eliminados de forma definitiva.